In [1]:
#coding: utf-8
import os
import time
import threading
from queue import Queue

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from googletrans import Translator

import pyautogui

In [2]:
other_log_id = -1
other_msgs = []
my_msgs = []

woo_url = "https://wootalk.today"
driver_name = 'chromedriver83.0.4103.39.exe'
path = os.path.join('driver', driver_name)
woo_url = "https://wootalk.today"


In [3]:
chrome_options = Options()
driver = webdriver.Chrome(executable_path=path,
                          options=chrome_options)   
baha_url = "https://forum.gamer.com.tw/C.php?bsn=60076&snA=5808611&tnum=1"
driver.get(woo_url)

In [8]:
def synchronized(func):

    func.__lock__ = threading.Lock()

    def synced_func(*args, **kws):
        with func.__lock__:
            return func(*args, **kws)

    return synced_func

class setInterval:
    def __init__(self, interval, action, fargs=(), fkwargs={}, *targs, **tkwargs) :
        self.interval = interval
        self.action = action
        self.event = threading.Event()
        thread = threading.Thread(target=self.__setInterval,
                                  args=fargs, kwargs=fkwargs,
                                  *targs, **tkwargs)
        thread.start()

    def __setInterval(self, *args, **kwargs) :
        nextTime= time.time() + self.interval
        while not self.event.wait(nextTime - time.time()):
            nextTime += self.interval
            self.action(*args, **kwargs)

    def cancel(self) :
        self.event.set()



In [5]:
def extract_text(msg):
    msg = msg.split('陌生人：');
    msg = msg[1:]
    msg = '陌生人：'.join(msg);
    msg = msg.split(' (');
    msg.pop();
    msg = ' ('.join(msg);
    return msg


    
def send_message(msg):
    textField = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, 'messageInput')))
    textField.send_keys(msg)
    textField.send_keys(Keys.RETURN)

In [100]:
class MessageHandler:
    script = """
get_text = () => {
    let otherTexts = document.querySelectorAll('.stranger.text');
    let msgs = []
    let mids = []
    function extractText(msg) {
        msg = msg.textContent.split('陌生人：');
        msg.shift();
        msg = msg.join('陌生人：');
        msg = msg.split(' (');
        msg.pop();
        msg = msg.join(' (');
        return msg;
    }

    for (let otherText of otherTexts) {
        msgs.push(extractText(otherText))
        mids.push(parseInt(otherText.getAttribute('mid')))
    }
    return [msgs, mids];
}

_clearInput = () => {
    messageInput.value = '';
}

send = (msg) => {
    if (document.querySelector('#sendButton input').value != "回報") {
        messageInput.value = msg;
        sendMessage();
    }
}
"""
    _instance = None
    _no_init = False
    def __new__(cls, *args, **kwargs): 
        if cls._instance is None: 
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, driver, force_init=False):
        if self._no_init and not force_init:
            return;
        MessageHandler._no_init = True

        self.driver = driver
        self.curr_index = 0
        self.curr_msg = None
        self.other_log_mid = -1
        self.other_msgs = []
        self.others_msgs = []
        
        self.__actions = {
            'echo': self.echo,
            'tran': self.tran,
        }
        self.driver.execute_script(self.script)
        self.translator = Translator()
        
        if not hasattr(self, 'action_interval'):
            self.action_interval = None
        if not hasattr(self, 'auto_record_interval'):
            self.auto_record_interval = None

    def update_msgs(self):
        otherTexts, other_mids = self.driver.execute_script("return get_text();");

        for i, otherText in enumerate(otherTexts):
            if self.other_log_mid < other_mids[i]:
                self.other_msgs.append(otherText)
                self.other_log_mid = other_mids[i]
        
        if self.curr_index < len(self.other_msgs):
            self.curr_msg = self.other_msgs[self.curr_index]

    @synchronized
    def action(self, method, *args, **kwargs):
        assert method in self.__actions, "Method '%s' does not exists" % method

        # the method to handle message
        self.update_msgs()
        if self.curr_index < len(self.other_msgs):
            res_msg = self.__actions[method](*args, **kwargs)
            self.send_message(res_msg)
            self.curr_index += 1
        
    def send_message(self, msg):
        self.driver.execute_script('_clearInput()')
        text_field = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.ID, 'messageInput')))
        text_field.send_keys(msg)
        text_field.send_keys(Keys.RETURN)
        
    def echo(self):
        return self.curr_msg
    
    @synchronized
    def tran(self, dest='en'):
        return self.translator.translate(self.curr_msg, dest=dest).text
    
    @synchronized
    def record(self):
        if len(self.other_msgs) != 0 and self.driver.execute_script('return isLeave()'):
            self.others_msgs.append(self.other_msgs)
            self.curr_index = 0
            self.curr_msg = None
            self.other_log_mid = -1
            self.other_msgs = []
        
    def action_trigger(self, method, fargs=(), fkwargs={}, interval=0.1, *targs, **tkwargs):
        if not self.action_interval:
            self.action_interval = setInterval(interval, self.action, fargs=(method, *fargs), fkwargs=fkwargs, name=f'{method}')
            print(f'start auto {method}')
        else:
            self.action_interval.cancel()
            self.action_interval = None
            print(f'stop auto {method}')
            
    def record_trigger(self):
        if not self.auto_record_interval:
            self.auto_record_interval = setInterval(2, self.record, name='record')
            print(f'start auto record')
        else:
            self.auto_record_interval.cancel()
            self.auto_record_interval = None
            print(f'stop auto record')


class WooControler:
    script = """
    
isLeave = () => {
    let systemTexts = document.querySelectorAll('.system.text');
    if (systemTexts.length <= 1) {
        return true;
    }
    
    systemTexts = systemTexts[systemTexts.length-2].textContent.split("離開");
    if (systemTexts.length >= 2 && systemTexts[1] == "了，請按") {
        return true;
    } else {
        return false;
    }
}

sendFirstMsg = (msg) => {
    if (!msg) {
        return;
    }
    messageInput.value = msg;
    let self = this
    if (document.querySelectorAll('.me.text').length  == 0) {
        sendMessage();
        setTimeout(() => {
            self.sendFirstMsg(msg)
        }, 500)
    }
}

restart = (msg) => {
    let systemTexts = document.querySelectorAll('.system.text');
    if (systemTexts.length <= 1) {
        clickStartChat();
        sendFirstMsg(msg)
    } else {
        let checkleave = systemTexts[systemTexts.length-2].textContent.split("離開");
        if (checkleave.length >= 2 && checkleave[1] == "了，請按") {
            leave();
        }
    }
}


    """
    _instance = None
    _no_init = False
    def __new__(cls, *args, **kwargs): 
        if cls._instance is None: 
            cls._instance = super().__new__(cls) 
        return cls._instance

    def __init__(self, driver, force_init=False):
        if self._no_init and not force_init:
            return;
        WooControler._no_init = True

        self.driver = driver
        self.driver.execute_script(self.script)
        if not hasattr(self, 'self.auto_restart_interval'):
            self.auto_restart_interval = None

        
    def click_start(self):
        start_btn = WebDriverWait(self.driver, 3).until(
                    EC.presence_of_element_located((By.ID, 'startButton')))
        ActionChains(self.driver).move_to_element_with_offset(start_btn, 10, 10).click().perform()

    def click_leave(self):
        # Not working
        leave_btn = WebDriverWait(self.driver, 3).until(
                    EC.presence_of_element_located((By.ID, 'changeButton')))
        ActionChains(self.driver).move_to_element_with_offset(leave_btn, 10, 10).click().perform()
        popyes = WebDriverWait(self.driver, 30).until(
                    EC.presence_of_element_located((By.ID, 'popup-yes')))
        ActionChains(self.driver).move_to_element_with_offset(popyes, 10, 10).click().perform()

    def restart_trigger(self, msg):
        if not self.auto_restart_interval:
            self.auto_restart_interval = setInterval(2, self.driver.execute_script, fargs=(f'restart("{msg}")',), name='restart')
            print('start auto restart')
        else:
            self.auto_restart_interval.cancel()
            self.auto_restart_interval = None
            print('stop auto restart')

    def leave(self):
        self.driver.execute_script('leave()')
    
    def is_leave(self):
        return self.driver.execute_script('return isLeave()')

In [101]:
msgHl = MessageHandler(driver)
wooCt = WooControler(driver)

In [113]:
wooCt.restart_trigger('你好，我是自動回覆機器人第二版，我能加你所說的話翻成英文，雖然一樣沒用，但希望對您能有幫助')

stop auto restart


In [114]:
msgHl.record_trigger()

stop auto record


In [115]:
msgHl.action_trigger('tran')

stop auto tran


In [117]:
print(msgHl.other_msgs)
print(msgHl.others_msgs)

[]
[['真的嗎', '太酷了吧!', '可以跟我交往嘛'], ['真的嗎', '太酷了吧!', '可以跟我交往嘛']]


In [116]:
threading.enumerate()

[<_MainThread(MainThread, started 24904)>,
 <Thread(Thread-4, started daemon 26656)>,
 <Heartbeat(Thread-5, started daemon 21608)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 29540)>,
 <ParentPollerWindows(Thread-3, started daemon 27100)>]